# Working with Raster Bands

## Import Packages & Load Data

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In addition to Rasterio, we'll be working with NumPy, which is a useful package for working with arrays, like our images. We will also be using Matplotlib's Pyplot module to visualize our data.

In [ ]:
image_file = "/Users/mansi/Downloads/20170831_172754_101c_3b_Visual.tif"

In [ ]:
satdat = rasterio.open(image_file)

# satdat is our open dataset object
print(satdat)

## Examining Metadata

In [ ]:
# Get the image's coordinate reference system

satdat.crs

Because this image's CRS is in UTM, we know the projected units are in meters.

In [ ]:
# Minimum bounding box in projected units (meters)


print(satdat.bounds)

In [ ]:
# Get dimensions, in projected units (using the example GeoTIFF, that's meters)

width_in_projected_units = satdat.bounds.right - satdat.bounds.left
height_in_projected_units = satdat.bounds.top - satdat.bounds.bottom

print("Width: {}, Height: {}".format(width_in_projected_units, height_in_projected_units))

In [ ]:
# Number of rows and columns (pixels)

print("Rows: {}, Columns: {}".format(satdat.height, satdat.width))

In [ ]:
# This dataset's projection uses meters as projected units.  What are the dimensions of a single pixel in meters?

xres = (satdat.bounds.right - satdat.bounds.left) / satdat.width
yres = (satdat.bounds.top - satdat.bounds.bottom) / satdat.height

print(xres, yres)
print("Are the pixels square: {}".format(xres == yres)) #sanity check

In [ ]:
# Convert pixel coordinates on the image to geographic coordinates on the globe

# Upper left pixel
row_min = 0
col_min = 0

# Lower right pixel.  Rows and columns are zero indexing.
row_max = satdat.height - 1
col_max = satdat.width - 1

# Transform coordinates with the dataset's affine transformation.
topleft = satdat.transform * (row_min, col_min)
botright = satdat.transform * (row_max, col_max)

# These are UTM coordinates, not lat/long. UTM coordinates are printed with a cardinal direction (E/W, N/S)
print("Top left corner coordinates: {}".format(topleft)) 
print("Bottom right corner coordinates: {}".format(botright))

In [ ]:
# All of the metadata required to create an image of the same dimensions, datatype, format, etc. is stored in
# the dataset's profile:

satdat.profile

## Composing a scene

In [ ]:
# Again, let's examine band count and indexes to ensure we have a 3-band visual image
print(satdat.count)
print(satdat.indexes)

Because we know we're look at a PlanetScope 3-band visual satellite image here, we can define the bands by their order:

In [ ]:
# PlanetScope 3-band band order: BGR

blue, green, red = satdat.read()

In [ ]:
# To construct a visual image, we will need the red, green, and blue bands

visual_image = np.dstack((red, green, blue))

## Visualizing an Image

In [ ]:
plt.figure(0)
plt.imshow(visual_image)
plt.title("Visual Image")

Congratulations!! Here we see an image of Houston during floods caused by Hurricane Harvey in August of 2017.